In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import pickle
from tqdm import tqdm

In [3]:
from data.data_preparation import check_consistence, load_data, graph_preprocess, get_partitions

In [4]:
with open('/nfs/students/qian/papers100m_splits.pkl', 'rb') as handle:
    (train_indices, val_indices, test_indices) = pickle.load(handle)

In [5]:
adj_t = torch.load('/nfs/students/qian/data/papers100m/adj.pt')

In [6]:
scipy_adj = adj_t.to_scipy('csr')
adj_t = None

# val

In [9]:
import time
from neighboring.pernode_ppr_neighbor import topk_ppr_matrix


In [ ]:

s = time.time()

eps = 2e-5
ppr_mat, neighbors = topk_ppr_matrix(scipy_adj, 0.05, eps, val_indices[:100], topk=128, normalization='sym')

print(time.time() - s)
# with open('/nfs/students/qian/papers100m_val_pprmat.pkl', 'rb') as handle:
#     ppr_mat = pickle.load(handle)

In [ ]:
ppr_mat

In [ ]:
from batching.loader_prime_orient_ppr import ppr_fixed_loader as prime_ppr_loader

In [ ]:
s = time.time()

val_loader = prime_ppr_loader(ppr_mat[:, val_indices], 
                                     val_indices, 
                                     neighbors, 
                                     10000)

print(time.time() - s)

# test

In [ ]:
s = time.time()

eps = 1e-4
test_mat, test_nei = topk_ppr_matrix(scipy_adj, 0.05, eps, test_indices, topk=32, normalization='sym')

print(time.time() - s)

# with open('/nfs/students/qian/papers100m_test_pprmat.pkl', 'rb') as handle:
#     ppr_mat = pickle.load(handle)

In [ ]:
test_mat

In [ ]:
s = time.time()

test_loader = prime_ppr_loader(test_mat[:, test_indices], 
                                     test_indices, 
                                     test_nei, 
                                     10000)

print(time.time() - s)

# train

In [7]:
mask = np.random.randint(low=0, high=len(train_indices), size=300)

In [11]:
s = time.time()

eps = 2e-5
train_mat, train_neighbors = topk_ppr_matrix(scipy_adj, 0.05, eps, train_indices[mask], topk=128, normalization='sym')

print(time.time() - s)
# with open('/nfs/students/qian/papers100m_train_pprmat.pkl', 'rb') as handle:
#     ppr_mat = pickle.load(handle)

161.02235460281372


In [12]:
train_mat

<300x111059956 sparse matrix of type '<class 'numpy.float32'>'
	with 37952 stored elements in Compressed Sparse Row format>

In [13]:
37952 / 300

126.50666666666666

In [ ]:
s = time.time()

train_loader = prime_ppr_loader(train_mat[:, train_indices], 
                                     train_indices, 
                                     train_neighbors, 
                                     5000)

print(time.time() - s)

In [ ]:
711.860222 + \
78.45057272911072 +  1.3239221572875977 + \
88.06589150428772 + 2.0748612880706787 + \
268.18696808815 + 12.744482040405273

In [ ]:
ps = []
for p, n in train_loader:
    print(len(p), len(n))
    ps.append(p)

In [ ]:
ps = np.concatenate(ps)

In [ ]:
len(ps) == len(train_indices)

In [ ]:
set(ps) == set(train_indices)

In [ ]:
from scipy.sparse import find

In [ ]:
row, col, val = find(ppr_mat)

In [ ]:
_row = torch.sort(torch.from_numpy(row)).values

In [ ]:
import torch_scatter

In [ ]:
deg = torch_scatter.segment_coo(torch.ones_like(_row), _row.long())

In [ ]:
import matplotlib.pyplot as plt

plt.hist(deg.numpy())

In [ ]:
thresh = 1e-4
num_parts = 48

In [ ]:
mask = val > thresh

In [ ]:
mask = np.unique(col[mask])

In [ ]:
mask = torch.from_numpy(mask).long()

In [ ]:
adj_t = adj_t[mask, :][:, mask]

In [ ]:
adj_t

In [ ]:
_, partptr, perm = adj_t.partition(num_parts=num_parts, recursive=False, weighted=False)

In [ ]:
mask = mask.numpy()

In [ ]:
partitions = []

for i in range(len(partptr) - 1):
    partitions.append(mask[perm[partptr[i] : partptr[i + 1]].cpu().detach().numpy()])

In [ ]:
partitions

In [ ]:
primes = []

for part in partitions:
    primes.append(np.intersect1d(test_indices, part))

In [ ]:
seconds = []

for prime in tqdm(primes):
    second = set()
    for p in prime:
        pos = (test_indices == p).nonzero()[0]
        cols, vals = find(ppr_mat[pos, :])[1:]
        topks = cols[np.argsort(vals)[-64:]]
        second.update(topks)
    
    second = np.array(list(second), dtype=np.int64)
    globes = ppr_mat[np.in1d(test_indices, prime), :].sum(0).A1
    ind_globes = globes.nonzero()[0]
    val_globes = globes[ind_globes]
    mask = np.argpartition(val_globes, kth=max(0, len(val_globes) - 400000))[-400000:]
    second = np.union1d(second, ind_globes[mask])
    
    seconds.append(second)

In [ ]:
batches = []

In [ ]:
for p, n in zip(primes, seconds):
    assert np.all(np.in1d(p, n))
    batches.append((p, n))

In [ ]:
with open('/nfs/students/qian/papers100m_test_LBMB.pkl', 'wb') as handle:
    pickle.dump(batches, handle, protocol=pickle.HIGHEST_PROTOCOL)